In [1]:
from re import T
import re
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import os
import json
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

import helpers
import api

In [2]:
df = pd.read_csv('all_provas_enem_areas_sys_solved.csv')
df_MCQ = pd.DataFrame(columns=['area', 'year', 'context', 'question', 'options', 'correct_answer', 'bloom_level', 'difficult_level', 'relevance', 'adherence', 'answerability', 'correctness', 'feedback', 'rouge', 'bleu', 'system_answer', 'steps_answer', 'model'])
df_MCQ_EN = pd.DataFrame(columns=['area', 'year', 'context', 'question', 'options', 'correct_answer', 'bloom_level', 'difficult_level', 'relevance', 'adherence', 'answerability', 'correctness', 'feedback', 'rouge', 'bleu', 'system_answer', 'steps_answer', 'model'])
df_2013 = df.loc[(df['year']==2013) & (df['image'] == 'No')].head(1).copy()
df_2013['steps_answer'] = "" 
df_2013_EN = pd.DataFrame(columns=df_2013.columns)

In [5]:
for model in ["gpt-3.5-turbo"]:#, "gpt-3.5", "gpt-4-32k", "gpt-4"]:
    c=0
    gpt = api.GPT(model=model)
    for index, row in tqdm(df_2013.iterrows()):
        try:
            c+=1
            print(c)
            system_context, user_prompt = helpers.prompt_builder_PT([[row['area'], row['context']]])
            messages=[{"role":"system","content": system_context},
                  {"role":"user","content": user_prompt}]
            res = gpt.chat(messages)
            response = json.loads(res)
            for question in response['questoes'][0]['perguntas']:
                evals = helpers.evaluate_generated_mcq(response['questoes'][0]['context'], question['pergunta'])
                dic = {'area': row['area'], 'year': row['year'], 'context':row['context']}
                dic['question'] = question['pergunta']
                options = [{k: v for k, v in dict(option).items() if k != 'correta'} for option in question['options']]
                options =  dict(item for d in options for item in d.items())
                dic['options'] = str(options)
                dic['correct_answer'] = question['correct_answer'][0]
                dic['bloom_level'] = question['bloom_level']
                dic['difficult_level'] = question['difficult_level']
                eval_pt = question
                eval_pt['area'] = response['questoes'][0]['area']
                eval_pt['context'] = response['questoes'][0]['context']

                system_context, user_prompt = helpers.evaluator_builder_PT_(eval_pt)
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_eval = json.loads(gpt.chat(messages))
                dic['relevance'] = response_eval['relevance']
                dic['adherence'] = response_eval['adherence']
                dic['answerability'] = response_eval['answerability']
                dic['correctness'] = response_eval['correctness']
                dic['feedback'] = response_eval['feedback']
                dic['rouge'] = str(evals[0]['rougeL'])
                dic['bleu'] = evals[1]

                system_context, user_prompt = helpers.solve_builder_PT_([response['questoes'][0]['context'], question['pergunta'], str(options)])
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_answer = json.loads(gpt.chat(messages))
                dic['system_answer'] = '{'+ response_answer['answer_letter'] + ': ' + response_answer['answer_text']+'}'
                dic['steps_answer'] = response_answer['steps_answer']
                dic['model'] = model
        
                df_MCQ.loc[len(df_MCQ)] = dic
                #############################
            
                system_context, user_prompt = helpers.translate_PT_EN([row['area'], row['context'], question['pergunta'], str(options), question['correct_answer'][0], question['bloom_level'], question['difficult_level']])
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_translation = json.loads(gpt.chat(messages))
                dic_en = {'area': response_translation['1'], 'year': row['year'], 'context': response_translation['2'], 'question': response_translation['3'], 'options': response_translation['4'], 'correct_answer': response_translation['5'], 'bloom_level': response_translation['6'], 'difficult_level':response_translation['7']}
                
                system_context, user_prompt = helpers.evaluator_builder_EN([response_translation['1'], response_translation['2'], response_translation['3'], response_translation['4'], response_translation['6'], response_translation['7']])
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                response_eval_EN = json.loads(gpt.chat(messages))
                dic_en['relevance'] = response_eval_EN['relevance']
                dic_en['adherence'] = response_eval_EN['adherence']
                dic_en['answerability'] = response_eval_EN['answerability']
                dic_en['correctness'] = response_eval_EN['correctness']
                dic_en['feedback'] = response_eval_EN['feedback']
                evals_EN = helpers.evaluate_generated_mcq(response_translation['2'], response_translation['3'])
                dic_en['rouge'] = str(evals_EN[0]['rougeL'])
                dic_en['bleu'] = evals_EN[1]
                
                system_context, user_prompt = helpers.solve_builder_EN([response_translation['2'], response_translation['3'], response_translation['4']])
                messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
                ans = gpt.chat(messages)#.replace('"', '')
                response_answer_EN = json.loads(gpt.chat(messages))#ast.literal_eval(json.dumps(ans))#json.loads(re.sub(r"\'(.*?)\'", r'"\1"', ans))
                dic_en['system_answer'] = '{'+ response_answer_EN['answer_letter'] + ': ' + response_answer_EN['answer_text']+'}'
                dic_en['steps_answer'] = response_answer_EN['steps_answer']
                dic_en['model'] = model
                
                df_MCQ_EN.loc[len(df_MCQ_EN)] = dic_en
    
            system_context, user_prompt = helpers.solve_builder_PT_([row['context'], row['question'], row['options']])
            messages=[{"role":"system","content": system_context},
                  {"role":"user","content": user_prompt}]
            response_resposta = json.loads(gpt.chat(messages))
            df_2013.loc[index, 'system_answer'] = '{'+ response_resposta['answer_letter'] + ': ' + response_resposta['answer_text']+'}'
            df_2013.loc[index, 'steps_answer'] = response_resposta['steps_answer']
            
            ##############################
            dic_all_en = {'id': df_2013.loc[index, 'id'], 'year': df_2013.loc[index, 'year'], 'CE': df_2013.loc[index, 'CE'], 
                          'DS': df_2013.loc[index, 'DS'], 'EK': df_2013.loc[index, 'EK'], 'IC': df_2013.loc[index, 'IC'],
                          'MR': df_2013.loc[index, 'MR'], 'TC': df_2013.loc[index, 'TC'],
                         'image': df_2013.loc[index, 'image'], 'IU': df_2013.loc[index, 'IU'], 'ledor': df_2013.loc[index, 'ledor'], 
                          'figures': df_2013.loc[index, 'figures']
                         }
            system_context, user_prompt = helpers.translate_PT_EN([df_2013.loc[index, 'area'], df_2013.loc[index, 'context'], df_2013.loc[index, 'question'], df_2013.loc[index, 'options'], df_2013.loc[index, 'correct_answer'], df_2013.loc[index, 'cor']])
            messages=[{"role":"system","content": system_context},
                      {"role":"user","content": user_prompt}]
            print(json.loads(gpt.chat(messages)))
            ans = gpt.chat(messages).strip().replace('\n', '')
            response_translation_EN = ast.literal_eval(ans)
            dic_all_en['area'] = response_translation_EN['1']
            dic_all_en['context'] = response_translation_EN['2']
            dic_all_en['question'] = response_translation_EN['3']
            dic_all_en['options'] = response_translation_EN['4']
            dic_all_en['correct_answer'] = response_translation_EN['5']
            dic_all_en['cor'] = response_translation_EN['6']
            
            system_context, user_prompt = helpers.solve_builder_EN([dic_all_en['context'], dic_all_en['question'], dic_all_en['options']])
            messages=[{"role":"system","content": system_context},
                  {"role":"user","content": user_prompt}]
            response_resposta_EN = json.loads(gpt.chat(messages))
            dic_all_en['system_answer'] = '{'+ response_resposta_EN['answer_letter'] + ': ' + response_resposta_EN['answer_text']+'}'
            dic_all_en['steps_answer'] = response_resposta_EN['steps_answer']
            
            df_2013_EN.loc[len(df_2013_EN)] = dic_all_en
            
        except NameError as errc:
            print(errc)

0it [00:00, ?it/s]

1
{
    "answer_letter": "A",
    "answer_text": "Meet the needs of work, supply, transportation, health, energy, and water of the population.",
    "steps_answer": "The main objective of the gigantic movement of city construction mentioned in the text is to meet the needs of work, supply, transportation, health, energy, and water of the population. This is evident from the description provided in the text."
}
{'answer_letter': 'A', 'answer_text': 'Meet the needs of work, supply, transportation, health, energy, and water of the population.', 'steps_answer': 'The main objective of the gigantic movement of city construction mentioned in the text is to meet the needs of work, supply, transportation, health, energy, and water of the population. This is evident from the statement that the territory was occupied and the conditions for living in this space were built, even though the urban growth direction has not fully met all these needs.'}
{
    "answer_letter": "B",
    "answer_text": "Co

1it [01:23, 83.54s/it]


In [9]:
df_MCQ

,area,year,context,question,options,correct_answer,bloom_level,difficult_level,relevance,adherence,answerability,correctness,feedback,rouge,bleu,system_answer,steps_answer,model
0,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,2013,\nTrata-se de um gigantesco movimento de const...,Qual é a principal finalidade do gigantesco mo...,{'A': 'Atender às necessidades de trabalho da ...,{'C': 'Proporcionar o assentamento residencial...,Compreensão,Iniciante,5,4,5,1,"A questão é altamente relevante, pois está dir...","Score(precision=0.4666666666666667, recall=0.1...",1.464796e-02,{C: Proporcionar o assentamento residencial da...,A principal finalidade do gigantesco movimento...,gpt-3.5-turbo
1,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,2013,\nTrata-se de um gigantesco movimento de const...,Qual é a principal razão para o gigantesco mov...,"{'A': 'Atender às necessidades de trabalho, ab...","{'A': 'Atender às necessidades de trabalho, ab...",Lembrar,Iniciante,5,4,5,1,"A questão é altamente relevante, pois está dir...","Score(precision=0.4117647058823529, recall=0.1...",1.972237e-02,"{A: Atender às necessidades de trabalho, abast...",A principal razão para o gigantesco movimento ...,gpt-3.5-turbo
2,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,2013,\nTrata-se de um gigantesco movimento de const...,Qual é um dos principais objetivos do gigantes...,"{'A': 'Atender às necessidades de trabalho, ab...","{'A': 'Atender às necessidades de trabalho, ab...",Lembrar,Iniciante,5,4,5,1,"A questão é altamente relevante, pois está dir...","Score(precision=0.5, recall=0.1176470588235294...",1.735789e-02,"{A: Atender às necessidades de trabalho, abast...",O texto menciona que o gigantesco movimento de...,gpt-3.5-turbo
3,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,2013,\nTrata-se de um gigantesco movimento de const...,O que foi construído no território devido ao m...,"{'A': 'Condições insalubres.', 'B': 'Condições...",{'B': 'Condições para viver naquele espaço.'},Compreender,Intermediário,5,5,5,1,"A questão apresenta alta relevância, pois está...","Score(precision=0.375, recall=0.08823529411764...",6.980008e-03,{B: Condições para viver naquele espaço.},"O texto menciona que, devido ao movimento de c...",gpt-3.5-turbo
4,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,2013,\nTrata-se de um gigantesco movimento de const...,Por que o crescimento urbano não respondeu sat...,"{'A': 'Falta de recursos financeiros.', 'B': '...",{'B': 'Falta de planejamento urbano adequado.'},Analisar,Avançado,5,5,5,1,A questão de múltipla escolha é altamente rele...,"Score(precision=0.6875, recall=0.1617647058823...",2.438358e-79,{B: Falta de planejamento urbano adequado.},"Para responder a essa pergunta, é necessário a...",gpt-3.5-turbo


In [10]:
df_MCQ_EN

,area,year,context,question,options,correct_answer,bloom_level,difficult_level,relevance,adherence,answerability,correctness,feedback,rouge,bleu,system_answer,steps_answer,model
0,HUMANITIES AND THEIR TECHNOLOGIES,2013,\nThis is a gigantic movement of city construc...,What is one of the main objectives of the giga...,"{'A': 'Meet the needs of work, supply, transpo...","{'A': 'Meet the needs of work, supply, transpo...",Remember,Beginner,5,4,5,0,The question is highly relevant to the context...,"Score(precision=0.4444444444444444, recall=0.1...",9.406458e-03,"{A: Meet the needs of work, supply, transporta...",The main objective of the gigantic movement of...,gpt-3.5-turbo
1,HUMANITIES AND THEIR TECHNOLOGIES,2013,\nThis is a gigantic movement of city construc...,What was built in the territory due to the cit...,"{'A': 'Unhealthy conditions.', 'B': 'Condition...",{'B': 'Conditions to live in that space.'},Understand,Intermediate,5,4,5,1,The question is highly relevant to the provide...,"Score(precision=0.3333333333333333, recall=0.0...",5.482474e-157,{B: Conditions to live in that space.},The text mentions that the territory was occup...,gpt-3.5-turbo
2,HUMANITIES AND THEIR TECHNOLOGIES,2013,\nThis is a gigantic movement of city construc...,Why did urban growth not satisfactorily meet a...,"{'A': 'Lack of financial resources.', 'B': 'La...",{'B': 'Lack of adequate urban planning.'},Analyzing,Advanced,5,4,5,1,The question is highly relevant to the context...,"Score(precision=0.5, recall=0.0845070422535211...",4.015334e-157,{B: Lack of adequate urban planning.},The passage mentions that urban growth did not...,gpt-3.5-turbo
